출처 : [PyTorch Transformer](https://pytorch.org/tutorials/beginner/basics/transforms_tutorial.html)

이 튜토리얼은 [nn.Transformer](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html) 모듈을 사용해서 seq2seq 학습에 대해 다룬다.

PyTorch 1.2부터 [Attention is All You Need](https://arxiv.org/pdf/1706.03762.pdf) 논문에 기반한 표준 transformer 모듈이 포함된다. Recurrent Neural Networks (RNNs)와 비교해서, transformer 모델이 seq2seq 작업에 더 효과적이라는 것이 증명되었다. `nn.Transformer` 모듈은 input과 output 사이의 전역 의존성을 이끌어내기 위해 attention mechchanism ([nn.MultiheadAttention](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html)에서 구현)을 사용한다. `nn.Transformer` 모듈은 고도로 모듈화가 되어 있어 단일 컴포넌트 (예, [nn.TransformerEncoder](https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoder.html))를 쉽게 조정하거나 구성할 수 있다.

![transformer](https://pytorch.org/tutorials/_images/transformer_architecture.jpg)

## Define the model
이 튜토리얼에서, language modeling 작업용으로 `nn.TransformerEncoder`를 학습한다. Language modeling 작업이란 주어진 단어(또는 단어 sequence)가 앞의 단어 sequence를 따를 가능성에 대한 확률을 구한다. 우선 토큰 sequence는 embedding layer로 전달된 다음, 단어 순서를 설명하기 위해 positional encoding layer로 전달된다 (자세한 설명은 다음 문단에). `nn.TransformerEncoder`는 여러 개의 [nn.TransformerEncoderLayer](https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoderLayer.html)로 이루어져 있다. input sequence와 함께 정사각형 attention mask가 필요한데, 왜냐하면 `nn.TransformerEncoder`의 self-attention layer는 input sequence의 앞부분에만 적용되기 때문이다. Language modeling 작업에서 뒷부분의 토큰은 마스킹되어야 한다. output 단어들의 확률분포를 생성하기 위해서는, `nn.TransformerEncoder`의 output은 linear layer를 통과한 후 log-softmax function을 거친다.

In [1]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output


def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

`PositionalEncoding` 모듈은 sequence에 있는 토큰들의 상대적이거나 절대적인 위치에 대한 정보를 주입한다. Positional encodings는 embeddings와 동일한 dimension을 갖고 있어서 같이 더할 수 있다. 아래의 코드는 `sin`, `cos` functions를 사용해서 positional encoding에 대한 구현이다.

In [2]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

## Load and batch data
이 튜토리얼은 Wikitext-2 dataset를 생성하기 위해 `torchtext`를 사용한다. torchtext datasets를 사용하기 위해서는 다음과 같은 명령어로 torchdata를 설치한다

In [ ]:
%%bash
pip install torchdata

vocab 객체는 train dataset을 기반으로 만들고 토큰을 수치화해서 tensor로 만들기위해 사용된다. Wikitext-2는 희귀한 토큰을 `<unk>`로 표현한다.

1-D vector로 이루어진 순서 데이터가 주어지면, `batchify()`는 데이터를 `batch_size` column 수만큼 배열한다. 만약 데이터가 `batch_size` column 수만큼 나누어 떨이지지 않는다면, 데이터를 자른다. 예를들어, 길이가 26인 알파벳 data가 있고 `batch_size=4`라면, 알파벳을 길이가 6인 sequences 4개를 만든다.

$$
\begin{bmatrix}
A & B & C & \cdots & X & Y & Z
\end{bmatrix}
\Rightarrow
\begin{bmatrix}
\begin{bmatrix}
A \\ B \\ C \\ D \\ E \\ F
\end{bmatrix}
\begin{bmatrix}
G \\ H \\ I \\ J \\ K \\ L
\end{bmatrix}
\begin{bmatrix}
M \\ N \\ O \\ P \\ Q \\ R
\end{bmatrix}
\begin{bmatrix}
S \\ T \\ U \\ V \\ W \\ X
\end{bmatrix}
\end{bmatrix}
$$

Batching은 병렬처리를 가능하게 하지만, 각 column을 독립척으로 처리한다. 그래서 `G`와 `F`의 의존성은 위의 예에서 학습할 수 없다.

In [3]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

# train_iter was "consumed" by the process of building the vocab,
# so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Args:
        data: Tensor, shape [N]
        bsz: int, batch size

    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

### Functions to generate input and target sequence
`get_batch()`는 transformer model을 위한 input-target sequences를 생성한다. `get_batch()`는 소스 데이터를 `bptt` 길이의 chunks로 세분화한다. Language modeling 작업을 위해, model은 `Target`이라는 미래 단어들이 필요하다. 예를들어, `bptt`가 2라면, `i=0`일 때의 2개의 미래 단어를 얻을 수 있다.

![matrix](https://pytorch.org/tutorials/_images/transformer_input_target.png)

(batch마다 2개의 단어를 묶어야 하므로 input (A,B), target (B,C), input (G, H), target (H, I), ... 로 row가 아니라 column으로 같이 묶여야 한다. 위의 행렬에 오류 있음.)

chunks는 데이터의 0번째 차원에 있다는 것을 유의해야한다. batch는 1번째 차원에 있다.

In [4]:
bptt = 35
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

## Initiate an instance
model hyperparameters는 아래에 정의되어 있다. vocab size는 vocab object와 같다.

In [5]:
ntokens = len(vocab)  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # number of heads in nn.MultiheadAttention
dropout = 0.2  # dropout probability
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

## Run the model
학습은 [SGD](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html) (stochastic gradient descent) optimizer를 사용하고 loss function으로 [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)를 사용한다. learning rate는 5.0으로 초기화하고 [StepLR](https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.StepLR.html)을 사용해 schedule을 한다. 학습하는 동안 [nn.utils.clip_grad_norm_](https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html)을 사용해 gradient exploding을 방지한다.

In [6]:
import copy
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        seq_len = data.size(0)
        if seq_len != bptt:  # only on last batch
            src_mask = src_mask[:seq_len, :seq_len]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            seq_len = data.size(0)
            if seq_len != bptt:
                src_mask = src_mask[:seq_len, :seq_len]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += seq_len * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

epoch를 돌면서 이때까지 본 것 중 validation loss가 가장 좋다면 모델을 저장한다. 또 각 epoch마다 learning rate를 조절한다.

In [7]:
best_val_loss = float('inf')
epochs = 3

with TemporaryDirectory() as tempdir:
    best_model_params_path = os.path.join(tempdir, "best_model_params.pt")

    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        train(model)
        val_loss = evaluate(model, val_data)
        val_ppl = math.exp(val_loss)
        elapsed = time.time() - epoch_start_time
        print('-' * 89)
        print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
            f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
        print('-' * 89)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_params_path)

        scheduler.step()
    model.load_state_dict(torch.load(best_model_params_path)) # load best model states

| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 26.08 | loss  8.23 | ppl  3737.76
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch 11.58 | loss  6.92 | ppl  1010.80
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch 11.46 | loss  6.47 | ppl   645.55
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch 11.28 | loss  6.32 | ppl   555.72
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch 11.52 | loss  6.20 | ppl   493.80
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch 11.46 | loss  6.17 | ppl   479.33
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch 11.43 | loss  6.12 | ppl   455.35
| epoch   1 |  1600/ 2928 batches | lr 5.00 | ms/batch 11.58 | loss  6.11 | ppl   451.57
| epoch   1 |  1800/ 2928 batches | lr 5.00 | ms/batch 11.61 | loss  6.04 | ppl   419.10
| epoch   1 |  2000/ 2928 batches | lr 5.00 | ms/batch 11.65 | loss  6.02 | ppl   412.79
| epoch   1 |  2200/ 2928 batches | lr 5.00 | ms/batch 11.67 | loss  5.90 | ppl   364.81
| epoch   1 |  2400/ 

## Evaluate the best model on the test dataset

In [8]:
test_loss = evaluate(model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

| End of training | test loss  5.53 | test ppl   252.38
